<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Inspecting-and-monitoring-deep-learning-models-using-Keras-callbacks-and-TensorBoard" data-toc-modified-id="Inspecting-and-monitoring-deep-learning-models-using-Keras-callbacks-and-TensorBoard-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Inspecting and monitoring deep-learning models using Keras callbacks and TensorBoard</a></span></li></ul></div>

# Inspecting and monitoring deep-learning models using Keras callbacks and TensorBoard

In [1]:
import keras

Using TensorFlow backend.


In [2]:
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 10000  # number of words to consider as features
max_len = 700  # cut texts after this number of words (among top max_features most common words)

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 700)
x_test shape: (25000, 700)


In [3]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

In [4]:
model = Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(1))

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 700, 128)          1280000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 694, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 138, 32)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 132, 32)           7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 1,315,937
Trainable params: 1,315,937
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [7]:
!mkdir -p my_log_dir

In [8]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor='acc', patience=2),
    keras.callbacks.TensorBoard(log_dir='my_log_dir', histogram_freq=1, embeddings_freq=1,)]

In [9]:
history = model.fit(x_train, y_train, 
                   epochs=10, 
                   batch_size=128,
                   validation_split=0.2,
                   callbacks=callbacks_list)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 46s 2ms/step - loss: 8.0792 - acc: 0.4987 - val_loss: 7.9785 - val_acc: 0.5050
Epoch 2/10
20000/20000 [==============================] - 43s 2ms/step - loss: 8.0792 - acc: 0.4987 - val_loss: 7.9785 - val_acc: 0.5050
Epoch 3/10
20000/20000 [==============================] - 45s 2ms/step - loss: 8.0792 - acc: 0.4987 - val_loss: 7.9785 - val_acc: 0.5050
